Consider the following problem:  you'd like to enable users to automatically extract a function from a Jupyter notebook and publish it as a service.  Actually serializing a closure from a function in a given environment is not difficult, given the [`cloudpickle`](https://github.com/cloudpipe/cloudpickle) module.  But merely constructing a serialized closure isn't enough, since in general this function may require other modules to be available to run in another context.  

Therefore, we need some way to identify the modules required by a function (and, ideally, the packages that provide these modules).  Since engineering time is limited, it's probably better to have an optimistic-and-imperfect estimate and allow users to override it (by supplying additional dependencies when they publish a function) than it is to have a sound and conservative module list.

The [`modulefinder`](https://docs.python.org/3.6/library/modulefinder.html) module in the Python standard library might initially seem like an attractive option, but it is unsuitable because it operates at the level of scripts.  In order to use `modulefinder` on a single function from a notebook, we'd either have an imprecise module list (due to running the whole notebook) or we'd need to essentially duplicate a lot of its effort in order to [slice backwards](https://en.wikipedia.org/wiki/Program_slicing) from the function invocation so we could extract a suitably pruned script.

Fortunately, you can interrogate nearly any aspect of any object in a Python program, and functions are no exception.  If we could inspect the captured variables in a closure, we could identify the ones that are functions and figure out which modules they were declared in.  That would look something like this:

In [1]:
import inspect

def module_frontier(f):
  worklist = [f]
  seen = set()
  mods = set()
  for fn in worklist:
    cvs = inspect.getclosurevars(fn)
    gvars = cvs.globals
    for k, v in gvars.items():
      if inspect.ismodule(v):
        mods.add(v.__name__)
      elif inspect.isfunction(v) and id(v) not in seen:
        seen.add(id(v))
        mods.add(v.__module__)
        worklist.append(v)
      elif hasattr(v, "__module__"):
        mods.add(v.__module__)
  return list(mods)

The [`inspect`](https://docs.python.org/3/library/inspect.html#inspect.Signature.bind) module provides a friendly interface to inspecting object metadata.  In the above function, we're constructing a worklist of all of the captured variables in a given function's closure.  We're then constructing a set of all of the modules directly or transitively referred to by those captured variables, whether these are modules referred to directly, modules declaring functions referred to by captured variables, or modules declaring other values referred to by captured variables (e.g., native functions).  Note that we add any functions we find to the worklist (although we don't handle `eval` or other techniques), so we'll capture at least some of the transitive call graph in this case.

This approach seems to work pretty sensibly on simple examples:

In [2]:
import numpy as np
from numpy import dot

def f(a, b):
    return np.dot(a, b)

def g(a, b):
    return dot(a, b)

def h(a, b):
    return f(a, b)

In [3]:
{k.__name__ : module_frontier(k) for k in [f,g,h]}

{'f': ['numpy.core.multiarray', 'numpy'],
 'g': ['numpy.core.multiarray'],
 'h': ['numpy.core.multiarray', '__main__', 'numpy']}

It also works on itself, which is a relief:

In [4]:
module_frontier(module_frontier)

['inspect']

While these initial experiments are promising, we shouldn't expect that a simple approach won't cover everything we might want to do.  Let's look at a (slightly) more involved example to see if it breaks down.

We'll use the k-means clustering implementation from scikit-learn to optimize some cluster centers in a model object.  We'll then capture that model object in a closure and analyze it to see what we might need to import to run it elsewhere.

In [5]:
from sklearn.cluster import KMeans
import numpy as np

data = np.random.rand(1000, 2)

model = KMeans(random_state=0).fit(data)

def km_predict_one(sample):
    sample = np.array(sample).reshape(1,-1)
    return model.predict(sample)[0]

In [6]:
km_predict_one([0.5, 0.5])

7

In [7]:
module_frontier(km_predict_one)

['sklearn.cluster.k_means_', 'numpy']

So far, so good.  Let's say we want to publish this simple model as a lighter-weight service (without a scikit-learn dependency).  We can get that by reimplementing the `predict` method from the k-means model:

In [8]:
centers = model.cluster_centers_
from numpy.linalg import norm

def km_predict_two(sample):
    _, idx = min([(norm(sample - center), idx) for idx, center in enumerate(centers)])
    return idx

In [9]:
km_predict_two([0.5, 0.5])

7

What do we get if we analyze the second method?

In [10]:
module_frontier(km_predict_two)

[]

_This is a problem!_  We'd expect that `norm` would be a captured variable in the body of `km_predict_two` (and thus that `numpy.linalg` would be listed in its module frontier), but that isn't the case.  We can inspect the closure variables:

In [11]:
inspect.getclosurevars(km_predict_two)

ClosureVars(nonlocals={}, globals={'centers': array([[ 0.84764105,  0.15126092],
       [ 0.76039891,  0.84267179],
       [ 0.14871645,  0.48878765],
       [ 0.504071  ,  0.19950246],
       [ 0.24086058,  0.83960717],
       [ 0.17965483,  0.14435561],
       [ 0.85896443,  0.48130862],
       [ 0.51427112,  0.58713655]])}, builtins={'min': <built-in function min>, 'enumerate': <class 'enumerate'>}, unbound=set())

We can see the cluster centers as well as the `min` function and the `enumerate` type.  But `norm` isn't in the list.  Let's dive deeper.  We can use the [`dis`](https://docs.python.org/3/library/dis.html) module to inspect the Python bytecode for a given function:

In [12]:
from dis import Bytecode
for inst in Bytecode(km_predict_two):
    print("%d: %s(%s)" % (inst.offset, inst.opname, inst.argrepr))

0: LOAD_GLOBAL(min)
2: LOAD_CLOSURE(sample)
4: BUILD_TUPLE()
6: LOAD_CONST(<code object <listcomp> at 0x1137fdd20, file "<ipython-input-8-5a350184a257>", line 5>)
8: LOAD_CONST('km_predict_two.<locals>.<listcomp>')
10: MAKE_FUNCTION()
12: LOAD_GLOBAL(enumerate)
14: LOAD_GLOBAL(centers)
16: CALL_FUNCTION()
18: GET_ITER()
20: CALL_FUNCTION()
22: CALL_FUNCTION()
24: UNPACK_SEQUENCE()
26: STORE_FAST(_)
28: STORE_FAST(idx)
30: LOAD_FAST(idx)
32: RETURN_VALUE()


Ah ha!  The body of our list comprehension, which contains the call to `norm`, is a separate code object that has been stored in a constant.  Let's look at the constants for our function:

In [13]:
km_predict_two.__code__.co_consts

(None,
 <code object <listcomp> at 0x1137fdd20, file "<ipython-input-8-5a350184a257>", line 5>,
 'km_predict_two.<locals>.<listcomp>')

We can see the code object in the constant list and use `dis` to disassemble it as well:

In [14]:
for inst in Bytecode(km_predict_two.__code__.co_consts[1]):
    print("%d: %s(%s)" % (inst.offset, inst.opname, inst.argrepr))

0: BUILD_LIST()
2: LOAD_FAST(.0)
4: FOR_ITER(to 30)
6: UNPACK_SEQUENCE()
8: STORE_FAST(idx)
10: STORE_FAST(center)
12: LOAD_GLOBAL(norm)
14: LOAD_DEREF(sample)
16: LOAD_FAST(center)
18: BINARY_SUBTRACT()
20: CALL_FUNCTION()
22: LOAD_FAST(idx)
24: BUILD_TUPLE()
26: LIST_APPEND()
28: JUMP_ABSOLUTE()
30: RETURN_VALUE()


Once we've done so, we can see that the list comprehension has loaded `norm` from a global, which we can then resolve and inspect:

In [15]:
km_predict_two.__globals__["norm"]

<function numpy.linalg.linalg.norm>

In [16]:
_.__module__

'numpy.linalg.linalg'

This is ongoing work and future notebooks will cover refinements to this technique.  It has been fun to learn about the power (and limitations) of the `inspect` module -- as well as a little more about how Python handles code blocks in list comprehension.